# Lecture 7. Data Wrangling: Join, Combine, and Reshape

In many applications, data may be spread across a number of files or databases or be arranged in a form that is not easy to analyze. Hence, we need to firstly combine, join, and rearrange data. 

In [182]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

---

## 7.3 Hierarchical Indexing

Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. 
 * It provides a way for you to work with higher dimensional data in a lower dimensional form. 
 
Let's create a Series with a **list of lists (or arrays)** as the index:

In [183]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [184]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

**Partial indexing for a hierarchically indexed object**

In [185]:
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [186]:
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [187]:
data[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [188]:
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

Selection is even possible from an **“inner”** level.

Suppose that you want to extract all the rows whose inner index values equal 2:

In [189]:
data[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [190]:
data.loc[:, 2]   # prefered

a    0.478943
c    0.092908
d    0.281746
dtype: float64

With a DataFrame, either axis can have a hierarchical index:

In [191]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects):

In [192]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With partial column indexing you can similarly select groups of columns:

In [193]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 7.3.1 Reordering and Sorting Levels

* At times you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. <br>
<br>
* The ```swaplevel``` takes two level numbers or names and returns a new object with the levels interchanged (***but the data is otherwise unaltered***). 

In [194]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [195]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

```sort_index``` sorts the data using only the values **in a single level**.

In [ ]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [197]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 7.3.2 Indexing with a DataFrame's columns

In [198]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s ```set_index``` function will create a new DataFrame using one or more of its columns as the index:

In [199]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [200]:
frame2.shape

(7, 2)

By default the columns are removed from the DataFrame, though you can leave them in:

In [201]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

```reset_index``` does the opposite of ```set_index```: the hierarchical index levels are moved into the columns:

In [202]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


---

## 7.4 Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:
* ```pandas.merge```: connects rows in DataFrames based on one or more keys. 
* ```pandas.concat```: concatenates or “stacks” together objects along an axis.
* The ```combine_first``` method enables splicing together overlapping data to fill in missing values in one object with values from another.

### 7.4.1 Database-Style DataFrame Joins

```Merge``` or ```join``` operations combine datasets by linking rows using one or more keys. These operations are central to e.g., SQL.

In [203]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [204]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a **many-to-one** join: The data in ```df1``` has multiple rows labeled a and b, whereas ```df2``` has only one row for each value in the ```key``` column.

In [205]:
pd.merge(df1, df2)    # By default, we use the inner join
                      # Therefore, we keep only the keys that appear in both df1 and df2

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


Note that I didn’t specify which column to join on. If that information is not specified, merge uses the overlapping column names as the keys. 

**It’s a good practice to specify explicitly, though:**

In [206]:
pd.merge(df1, df2, on='key')  

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


If the column names are different in each object, you can specify them separately:

In [207]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


* By default merge does an ```'inner'``` join; the keys in the result are the intersection, or the common set found in both tables. <br>
<br>
* Other possible options are ```'left'```, ```'right'```, and ```'outer'```. 
  * The outer join takes the union of the keys, combining the effect of applying both left and right joins.

In [208]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,a,2.0,0.0
1,a,4.0,0.0
2,a,5.0,0.0
3,b,0.0,1.0
4,b,1.0,1.0
5,b,6.0,1.0
6,c,3.0,NaN
7,d,NaN,2.0


**Many-to-many merges:**

In [209]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [210]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [211]:
pd.merge(df1, df2, on='key', how='left')

## Many-to-many joins form the Cartesian product of the rows. Since there were 
## three 'b' rows in the left DataFrame and two in the right one, 
## there are six 'b' rows in the result.

## how='left': we keep only the keys in df1.

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
...,...,...,...
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


To merge with **multiple keys**, pass a list of column names:

In [212]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,bar,one,3.0,6.0
1,bar,two,NaN,7.0
2,foo,one,1.0,4.0
3,foo,one,1.0,5.0
4,foo,two,2.0,NaN


A last issue to consider in merge operations is the treatment of overlapping column names. 

While you can address the overlap manually, merge has a ```suffixes``` option for specifying strings to append to overlapping names in the left and right DataFrame objects:

In [213]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [214]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 7.4.2 Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index. 

In this case, you can pass ```left_index=True``` or ```right_index=True``` (or both) to indicate that the index should be used as the merge key.

In [215]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [216]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [217]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


With hierarchically indexed data, things are more complicated, as joining on index is implicitly a **multiple-key merge**:

In [218]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [219]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [220]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [221]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [222]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [223]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a convenient join instance for merging by ```index```.
* In part for legacy reasons (i.e., much earlier versions of pandas), DataFrame’s ```join``` method performs a left join on the join keys, exactly preserving the left frame’s row index.

In [224]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [225]:
left1.join(right1, on='key')   # indicate that use 'key' of the left1 as index to merge with right1

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Lastly, for simple index-on-index merges, you can pass **a list of DataFrames** to join

In [226]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [227]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


#### When can't we use ```pd.join```?

In [228]:
# try df1.join(df2, on='key')

### 7.4.3 Concatenating Along an Axis

Another kind of data combination operation is referred to as concatenation, binding, or stacking. 

NumPy’s ```concatenate``` function can do this with NumPy arrays:

In [229]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [230]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

#### In the context of pandas objects such as Series and DataFrame, having labeled axes enable you to further generalize array concatenation (```pandas.concat``` function). 

In [231]:
# Suppose we have three Series with no index overlap:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [232]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

* By default concat works along ```axis=0```, producing another Series.  <br>
<br>
* If you pass ```axis=1```, the result will instead be a DataFrame (```axis=1``` is the columns):

In [233]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis. 

You can instead intersect them by passing ```join='inner'```.

In [234]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [235]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [236]:
pd.concat([s1, s4], axis=1, join='inner')  # 'f' and 'g' labels disappeared because of join='inner' option.

,0,1
a,0,0
b,1,1


Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the ```keys``` argument:

In [237]:
s1

a    0
b    1
dtype: int64

In [238]:
s2

c    2
d    3
e    4
dtype: int64

In [239]:
s3

f    5
g    6
dtype: int64

In [240]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])   
result   # create a hierarchical index on the concatenation axis

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In the case of combining Series along ```axis=1```, the ```keys``` become the DataFrame **column headers**

In [241]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [242]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [243]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [244]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

A last consideration concerns DataFrames in which the row index does not contain any relevant data:

In [245]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741


In [246]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614


In [247]:
pd.concat([df1, df2])

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
0,-1.021228,0.476985,NaN,3.248944
1,0.302614,-0.577087,NaN,0.124121


In [248]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


### ```concat``` function arguments

* ```objs``` List or dict of pandas objects to be concatenated; this is the only required argument
* ```axis``` Axis to concatenate along; defaults to 0 (along rows)
* ```join``` Either ```'inner'``` or ```'outer'``` (```'outer'``` by default); whether to intersection (inner) or union (outer) together indexes along the other axes
* ```keys``` Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can either be a list or array of arbitrary values, an array of tuples, or a list of arrays (if multiple-level arrays passed in levels)
* ```levels``` Specific indexes to use as hierarchical index level or levels if keys passed
* ```names``` Names for created hierarchical levels if ```keys``` and/or ```levels``` passed
* ```ignore_index``` Do not preserve indexes along concatenation ```axis```, instead producing a new ```range(total_length)``` index

### 7.4.4 Combining Data with Overlap

In [249]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [250]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

C:\Users\Hatim\AppData\Local\Temp\ipykernel_19104\293356008.py:3: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  b[-1] = np.nan


f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [251]:
np.where(pd.isnull(a), b, a)   # performs the array-oriented equivalent of an if-else expression

array([0. , 2.5, 2. , 3.5, 4.5, nan])

#### Series has a ```combine_first``` method, which performs the equivalent of this operation:

In [252]:
a.combine_first(b)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    NaN
dtype: float64

With DataFrames, ```combine_first``` does the same thing **column by column**

In [253]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [254]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [255]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


---

## 7.5 Reshaping and Pivoting

### 7.5.1 Reshaping with Hierarchical Indexing

There are two primary actions:
* (1) ```stack```: this “rotates” or pivots from the columns in the data to the rows; 
* (2) ```unstack```: this pivots from the rows into the columns.

In [256]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [257]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [258]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:

In [259]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [260]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


#### Unstacking might introduce missing data

In [261]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [262]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [263]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [264]:
data2.unstack().stack(dropna=False)

C:\Users\Hatim\AppData\Local\Temp\ipykernel_19104\3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

* When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result. <br>
<br>
* When calling stack, we can indicate the name of the axis to stack:

In [265]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [266]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [267]:
df.unstack('number')

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [268]:
df.unstack('state').stack('side')

C:\Users\Hatim\AppData\Local\Temp\ipykernel_19104\3997798123.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack('state').stack('side')


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### 7.5.2 Pivoting “Long” to “Wide” Format (*NOT REQUIRED*)

A common way to store multiple time series in databases and CSV is in so-called **long or stacked format**. 

In [269]:
data = pd.read_csv('data/macrodata.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/macrodata.csv'

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
# We will look at PeriodIndex a bit more closely.
# In short, it combines the year and quarter columns to create a kind of time interval type.

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [ ]:
periods

In [ ]:
columns

In [ ]:
data = data.reindex(columns=columns)
print(data.head())

In [ ]:
data.index = periods
print(data.head())

In [ ]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
print(ldata.head())

This is the so-called **long format** for multiple time series, or other observational data with two or more keys (here, our keys are date and item). Each row in the table represents a single observation.
* Data is frequently stored this way in relational databases like SQL. 

#### You can reshape the data via the ```pivot``` command. 

The first two values ```('date', 'item')``` passed are the columns to be used respectively as the row and column index,  then finally an optional value column ```'value'```) to fill the DataFrame.

In [ ]:
?pd.DataFrame.pivot

In [ ]:
pivoted = ldata.pivot(index='date', columns='item', values='value')
print(pivoted.head())

Suppose you had **two value columns** that you wanted to reshape simultaneously:

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [ ]:
pivoted = ldata.pivot(index='date', columns='item', values=['value', 'value2'])
pivoted[:5]

### 7.5.3 Pivoting “Wide” to “Long” Format (*NOT REQUIRED*)

An inverse operation to pivot for DataFrames is ```pandas.melt```. 

Rather than transforming one column into many in a new DataFrame, **it merges multiple columns into one**, producing a DataFrame that is longer than the input.

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

* When using ```pandas.melt```, we must indicate which columns (if any) are group indicators. <br>
<br>
* Let’s use 'key' as the only group indicator here:

In [ ]:
melted = pd.melt(df, ['key'])
melted

Using ```pivot```, we can reshape back to the original layout:

In [ ]:
reshaped = melted.pivot(index='key', columns='variable', values='value')
reshaped

Since the result of pivot creates an index from the column used as the row labels, we may want to use ```reset_index``` to move the data back into a column:

In [ ]:
reshaped.reset_index()  

You can also specify a subset of columns to use as value columns:

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])   # hence we ignore the value 'C'

---

## END